# Install Packages

In [12]:
# ! pip -q install openai agno
# ! pip install pandas
# ! pip install python-dotenv
# ! pip install duckduckgo-search
# ! pip install -U tavily-python
# ! pip install tantivy
# ! pip install openai newspaper4k lxml_html_clean


# Load Secrets

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv(dotenv_path='../backend/.env')  # Adjust if not running from 'notebooks/'


# Access the variables
openai_key = os.getenv("OPENAI_API_KEY")
gmail_user = os.getenv("GMAIL_ADDRESS")
gmail_pass = os.getenv("GMAIL_APP_PASSWORD")
tavily_key = os.getenv("TAVILY_API_KEY")



print("Loaded API Key:", bool(openai_key))  # Should print True if loaded


Loaded API Key: True


# Import Packages

In [2]:
import os
from dotenv import load_dotenv
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools import tool
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.tavily import TavilyTools


# Team Structure:

In [16]:
from agno.agent import Agent
from agno.team import Team
from agno.models.openai import OpenAIChat
from agno.tools.knowledge import KnowledgeTools
from agno.vectordb.lancedb import LanceDb, SearchType
from agno.embedder.openai import OpenAIEmbedder

# Initialize the LanceDb vector database (NO .load() required)
knowledge_base = LanceDb(
    table_name="knowledge",
    uri="tmp/lancedb",
    search_type=SearchType.hybrid,
    embedder=OpenAIEmbedder(id="text-embedding-3-small")
)

# Initialize the KnowledgeTools with the knowledge base
knowledge_tools = KnowledgeTools(
    knowledge=knowledge_base,
    think=True,
    search=True,
    analyze=True
)

# Email Specialist Agent
email_agent = Agent(
    model=OpenAIChat(id="gpt-4"),
    name="Email Specialist",
    tools=["send_email"],
    description="Handles email drafting and sending"
)

# Research Specialist Agent
research_agent = Agent(
    model=OpenAIChat(id="gpt-4"),
    name="Research Specialist",
    tools=["tavily_search"],
    description="Conducts web research"
)

# Knowledge Specialist Agent
rag_agent = Agent(
    model=OpenAIChat(id="gpt-4"),
    name="Knowledge Specialist",
    tools=[knowledge_tools],
    knowledge=knowledge_base,
    search_knowledge=True,
    description="Handles knowledge base queries and RAG operations"
)


# Team Configuration:

In [18]:
from agno.agent import Agent
from agno.team import Team
from agno.models.openai import OpenAIChat

# Assuming email_agent, research_agent, and rag_agent are already defined

team_lead = Team(
    name="Team Lead",
    mode="coordinate",
    model=OpenAIChat(id="gpt-4"),
    members=[email_agent, research_agent, rag_agent],
    description="Orchestrates the workflow and coordinates other agents",
    instructions=[
        "Analyze the user query and determine the tasks required.",
        "Delegate tasks to the appropriate specialists.",
        "Collect and synthesize the outputs from all agents into a cohesive response.",
        "Ensure the final response meets the highest quality standards."
    ],
    add_datetime_to_instructions=True,
    enable_agentic_context=True,
    share_member_interactions=True,
    show_members_responses=True,
    markdown=True
)


# Example Use

In [20]:
response = team_lead.run("Hello, can you tell me about the latest developments in AI/LLM?")
print(response)


TeamRunResponse(event='RunResponse', content="Here are the most recent advancements and breakthroughs in the fields of Artificial Intelligence (AI) and Large Language Models (LLM):\n\n1. **OpenAI Codex**: This powerful LLM translates natural language to code. It was first presented in June 2021 via the GitHub Copilot project and was made publicly available in August 2021. More about OpenAI Codex can be found [here](https://openai.com/research/codex).\n\n2. **InstructGPT**: Another initiative by OpenAI, InstructGPT is designed to follow instructions, answer questions, and provide explanations. The project is aimed at improving the utility and safety of LLMs. For more details, visit [here](https://openai.com/research/instructgpt).\n\n3. **GPT-3.5-turbo**: Launched in December 2021, GPT-3.5-turbo is one of OpenAI's most advanced LLMs. It has multiple applications, including email drafting, Python code writing, and tutoring in various subjects. You can find more information about it [here]

In [21]:
print(response.content)

Here are the most recent advancements and breakthroughs in the fields of Artificial Intelligence (AI) and Large Language Models (LLM):

1. **OpenAI Codex**: This powerful LLM translates natural language to code. It was first presented in June 2021 via the GitHub Copilot project and was made publicly available in August 2021. More about OpenAI Codex can be found [here](https://openai.com/research/codex).

2. **InstructGPT**: Another initiative by OpenAI, InstructGPT is designed to follow instructions, answer questions, and provide explanations. The project is aimed at improving the utility and safety of LLMs. For more details, visit [here](https://openai.com/research/instructgpt).

3. **GPT-3.5-turbo**: Launched in December 2021, GPT-3.5-turbo is one of OpenAI's most advanced LLMs. It has multiple applications, including email drafting, Python code writing, and tutoring in various subjects. You can find more information about it [here](https://platform.openai.com/docs/guides/chat/introd

In [ ]:
team_lead.print_response("Hello, can you tell me about the latest developments in AI/LLM?")


Output()